<h1>Cross Encoder Scores</h1>
Pre-computer question - answer cross encoder scores for use in L2R model

See more <a href = "https://huggingface.co/cross-encoder/qnli-distilroberta-base">here</a>

In [2]:
import sys
sys.path.insert(1, '../code')

In [31]:
from utils import * 

import os 
import torch
import pandas as pd 

from tqdm import tqdm 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

<h2>Load documents</h2>

In [32]:
comments = load_docs("../data/ask_reddit_posts_augmented.jsonl", text_key = "comments")
posts = load_docs("../data/ask_reddit_posts_augmented.jsonl", text_key = "post")

<h2>Load model</h2>

In [33]:
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/qnli-distilroberta-base')

In [34]:
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/qnli-distilroberta-base')

In [35]:
docids = [doc[0] for doc in comments]

In [39]:
features = tokenizer(['How many people live in Berlin?', 'What is the size of New York?'], ['Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.'],  padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = torch.nn.functional.sigmoid(model(**features).logits)
    print(scores)

tensor([[0.9928],
        [0.0015]])


In [ ]:
doc2text_score = {}
for docid in tqdm(docids): 
    post = posts[docid][1]
    comment = comments[docid][1]

    features = tokenizer(post, comment, padding = True, truncation = True, return_tensors = "pt")

    model.eval()
    with torch.no_grad():
        score = torch.nn.functional.sigmoid(model(**features).logits)
        doc2text_score[docid] = float(score[0][0])

    

  1%|▎                                        | 1285/200001 [01:01<2:33:54, 21.52it/s]